In [13]:
import pandas as pd
import os
import numpy as np
import folium
import h3
import matplotlib.pyplot as plt
from matplotlib.colors import rgb2hex
import branca

p = os.path.join
cmap = plt.get_cmap('hot_r')
df = pd.read_csv(p('..', 'prediction', 'final_prediction.csv'), index_col=0)

In [14]:
for (c, city_df) in df.groupby('city'):
    ring = {r for h in city_df.index for r in h3.k_ring(h, 3)}
    latlons = [h3.h3_to_geo(r) for r in ring]
    lat, lon = np.array(latlons).mean(axis=0)
    m = folium.Map(location=[lat, lon])
    m.fit_bounds(latlons)
    cmap_legend = branca.colormap.LinearColormap([cmap(i/10) for i in range(10)], vmin=city_df['NO2_adjusted'].min(), vmax=city_df['NO2_adjusted'].max())
    cmap_legend.caption = 'NO2 Level (ppb)'
    cmap_legend.add_to(m)

    for r in ring:
        try:
            level = city_df.loc[r]['NO2_adjusted']
        except KeyError:
            continue
        folium.Polygon(
            h3.h3_to_geo_boundary(r),
            tooltip=f'{np.round(level, 1)} ppb',
            fill_color= rgb2hex(cmap(level/city_df['NO2_adjusted'].max())), 
            color='black',
            fill_opacity= 0.6
        ).add_to(m)
    m.save(f'maps/{c}.html')